In [1]:
from nn.model import *
from nn.optimizer import GDOptimizer
from nn.loss import MSELoss
from nn.vector_net import *
from gradient.grad_vector import GradVector

BATCH_SIZE = 256
FEATURE_SIZE = 16
MAX_ITER = 1000

y_true = GradVector(np.random.rand(BATCH_SIZE, 1))
x = GradVector(np.random.rand(BATCH_SIZE, FEATURE_SIZE))

pipeline = [VectorLinearLayer(FEATURE_SIZE, FEATURE_SIZE * 2), VectorReluLayer(), VectorLinearLayer(FEATURE_SIZE * 2, FEATURE_SIZE), VectorReluLayer(), VectorLinearLayer(FEATURE_SIZE, 1)]
model = Pipeline(pipeline=pipeline)
optimizer = GDOptimizer(model.parameters(), lr=5e-5)
mse = MSELoss()

for i in range(MAX_ITER):
    y_pred = model.forward(x)
    loss = mse.loss(y_pred=y_pred, y_true=y_true)
    loss.backward()
    optimizer.optimize()
    loss.zero_grad()
    if i % 25 == 0:
        print(f"MSE Loss: {loss.val[0]} after {i} iters")


MSE Loss: [26.93005773] after 0 iters
MSE Loss: [16.14781875] after 25 iters
MSE Loss: [8.25719663] after 50 iters
MSE Loss: [0.23324729] after 75 iters
MSE Loss: [0.91261081] after 100 iters
MSE Loss: [0.60822137] after 125 iters
MSE Loss: [0.21224207] after 150 iters
MSE Loss: [0.12094358] after 175 iters
MSE Loss: [0.19081519] after 200 iters
MSE Loss: [0.23175909] after 225 iters
MSE Loss: [0.18533779] after 250 iters
MSE Loss: [0.11993552] after 275 iters
MSE Loss: [0.0978816] after 300 iters
MSE Loss: [0.09779091] after 325 iters
MSE Loss: [0.10334298] after 350 iters
MSE Loss: [0.10659513] after 375 iters
MSE Loss: [0.10489917] after 400 iters
MSE Loss: [0.10013782] after 425 iters
MSE Loss: [0.09374829] after 450 iters
MSE Loss: [0.0884493] after 475 iters
MSE Loss: [0.08620421] after 500 iters
MSE Loss: [0.08739964] after 525 iters
MSE Loss: [0.0917356] after 550 iters
MSE Loss: [0.09747443] after 575 iters
MSE Loss: [0.10152244] after 600 iters
MSE Loss: [0.10232967] after 62

In [4]:
import torch
from nn.vector_net import *
from nn.loss import MSELoss
import numpy as np
from gradient.grad_vector import GradVector

BATCH_SIZE = 1024
MAX_ITER = 100


actual = np.random.rand(BATCH_SIZE, 1)
x = np.random.rand(BATCH_SIZE, 8)
layer1 = VectorLinearLayer(8, 16)
layer2 = VectorLinearLayer(16, 1)

t_actual = torch.tensor(actual, requires_grad=True)
data = torch.tensor(x, requires_grad=True)

weights1 = torch.tensor(layer1.weights.val, requires_grad=True)
weights2 = torch.tensor(layer2.weights.val, requires_grad=True)
bias1 = torch.tensor(layer1.bias.val, requires_grad=True)
bias2 = torch.tensor(layer2.bias.val, requires_grad=True)

res = data @ weights1 + bias1
res = res @ weights2 + bias2
torch_mse = MSELoss.loss(res, t_actual)
torch_mse.backward()

res_gv = layer1(GradVector(x))
res_gv = layer2(res_gv)
gv_actual = GradVector(actual)
gv_mse = MSELoss.loss(res_gv, gv_actual)
gv_mse.backward()

In [5]:
weights1.grad.allclose(torch.tensor(layer1.weights.gradient))

True

In [6]:
bias1.grad.allclose(torch.tensor(layer1.bias.gradient))

True

In [9]:
r = torch.rand((16,8,8,3))
r = 

[[2.47616447]]